## Fichier Medecins Communes


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from cartiflette import carti_download
import plotly.express as px

## Import des données



In [ ]:
df_commune = pd.read_excel("Bases de données/medecins_commune_2025.xlsx")[4:]
df_commune.reset_index(drop=True, inplace=True)
df_commune.columns = ["Code commune INSEE", "Commune", "Médecins généralistes en 2025"]

df_commune.head()

In [ ]:
df_commune_dens = pd.read_excel("Bases de données/POPULATION_MUNICIPALE_COMMUNES_FRANCE.xlsx")
df_commune_dens = df_commune_dens[["dep", "codgeo", "libgeo", "p21_pop"]]
#df_commune.reset_index(drop=True, inplace=True)
df_commune_dens.columns = ["Département", "Code commune INSEE", "Commune", "Population Municipales en 2024"]
df_commune_dens = df_commune_dens.sort_values(by="Code commune INSEE")

df_commune_dens.head()

In [ ]:
def prepare_communes(df_commune_dens, df_commune):
    df = df_commune_dens.merge(
        df_commune[["Code commune INSEE", "Médecins généralistes en 2025"]],
        on="Code commune INSEE",
        how="left"
    )

    df["Médecins généralistes en 2025"] = (
        df["Médecins généralistes en 2025"]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .str.replace(r"[^0-9.]", "", regex=True)
        .replace("", "0")
        .astype(float)
    )

    df["Population Municipales en 2024"] = (
        df["Population Municipales en 2024"]
        .astype(str)
        .str.replace(" ", "", regex=False)
        .astype(float)
    )

    return df


prepare_communes(df_commune_dens,df_commune).head()

## Indicateur Accessibilité aux soins


In [ ]:
# Pour la pondérisation des communes par distance

def poids_distance(d):
    if d <= 5:
        return 1
    elif d <= 10:
        return 0.7
    elif d <= 15:
        return 0.4
    elif d <= 30:
        return 0.1
    else:
        return 0

In [ ]:
deps = gpd.read_file(
    "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
)

In [ ]:
voisins = {}

for _, dep in deps.iterrows():
    code = dep["code"]   # ✅ BON NOM DE COLONNE

    voisins_dep = deps[
        deps.geometry.touches(dep.geometry)
    ]["code"].tolist()

    voisins[code] = voisins_dep

In [ ]:
voisins["75"]

In [ ]:
df = prepare_communes(df_commune_dens, df_commune)

def apl_commune(code_commune, df_commune_dens, df_commune, voisins, poids_distance, df):
    """
    Calcule l'APL (proxy administratif) pour une commune donnée
    """

    # infos commune cible
    row_i = df[df["Code commune INSEE"] == code_commune]
    if row_i.empty:
        raise ValueError("Code commune inconnu")

    dep_i = row_i["Département"].values[0]

    # bassin départemental
    bassin_deps = [dep_i] + voisins.get(dep_i, [])

    # 2. Étape 1 : calcul des R_j pour toutes les communes du bassin
    R = {}

    for _, row_j in df[df["Département"].isin(bassin_deps)].iterrows():
        com_j = row_j["Code commune INSEE"]
        dep_j = row_j["Département"]

        # population concurrente
        denom = 0
        for dep_k in bassin_deps:
            pop_k = df[df["Département"] == dep_k]["Population Municipales en 2024"].sum()
            denom += pop_k * poids_distance(0 if dep_j == dep_k else 30)

        if denom > 0:
            R[com_j] = row_j["Médecins généralistes en 2025"] / denom
        else:
            R[com_j] = 0

    # 3. Étape 2 : APL de la commune i
    apl_i = 0
    for com_j, Rj in R.items():
        dep_j = df[df["Code commune INSEE"] == com_j]["Département"].values[0]
        d = 0 if dep_j == dep_i else 30
        apl_i += Rj * poids_distance(d)

    return apl_i

print(apl_commune(
            code_commune="27162",
            df_commune_dens=df_commune_dens,
            df_commune=df_commune,
            voisins=voisins,
            poids_distance=poids_distance,
            df
        ))


In [ ]:
def apl_toutes_les_communes(df_commune_dens, df_commune,voisins, poids_distance):
    """
    Renvoie un DataFrame communal avec une colonne APL,
    en s'appuyant sur prepare_communes et apl_commune.
    """
    df = prepare_communes(df_commune_dens, df_commune)
    apl_values = []

    for code in df["Code commune INSEE"]:
        apl = apl_commune(
            code_commune=code,
            df_commune_dens=df_commune_dens,
            df_commune=df_commune,
            voisins=voisins,
            poids_distance=poids_distance
        )
        apl_values.append(apl)

    df["APL"] = apl_values

    return df

df_communes_apl = apl_toutes_les_communes(
    df_commune_dens=df_commune_dens,
    df_commune=df_commune,
    voisins=voisins,
    poids_distance=poids_distance
)

df_communes_apl.head()

